# main

> Entry point of the ReadNext command line tool

In [ ]:
#| default_exp main

## Imports

The command line interface is using [typer](https://typer.tiangolo.com/), a library to build command line interfaces. We also use [arxiv](https://github.com/lukasschwab/arxiv.py) to query their search service to display the articles' titles from the list of IDs proposed by the system.

Otherwise, we import all the internal modules of the project used to implement the different commands of the CLI.

In [ ]:
#| exports

import arxiv
import chromadb
import os
import typer
from dotenv import load_dotenv
from readnext import __version__
from readnext.arxiv_categories import exists, main, sub
from readnext.arxiv_sync import sync_arxiv
from readnext.embedding import embed_category_papers, download_embedding_model, embedding_system
from readnext.personalize import get_personalized_papers, save_personalized_papers_in_zotero
from rich import print
from typing_extensions import Annotated

## Command line interface

In [ ]:
#| export

app = typer.Typer()

### version

The `version` command displays the current installed version of ReadNext.

In [ ]:
#| export

@app.command()
def version():
    """Get the current installed version of ReadNext"""
    print(f"Version: {__version__}")

You can get the version number of the ReadNext instance installed of your machine by running:

```bash
readnext version
```

### Configuration

Display the current configuration of ReadNext.

In [ ]:
#| export

@app.command()
def config():
    """Get the current configuration of ReadNext"""
    print(f"DOCS_PATH: {os.environ.get('DOCS_PATH')}")
    print(f"RECOMMENDATIONS_PATH: {os.environ.get('RECOMMENDATIONS_PATH')}")
    print(f"CHROMA_DB_PATH: {os.environ.get('CHROMA_DB_PATH')}")
    print(f"MODELS_PATH: {os.environ.get('MODELS_PATH')}")
    print(f"EMBEDDING_SYSTEM: {os.environ.get('EMBEDDING_SYSTEM')}")
    print(f"ZOTERO_LIBRARY_TYPE: {os.environ.get('ZOTERO_LIBRARY_TYPE')}")
    print(f"ZOTERO_API_KEY: {os.environ.get('ZOTERO_API_KEY')}")
    print(f"ZOTERO_LIBRARY_ID: {os.environ.get('ZOTERO_LIBRARY_ID')}")
    print(f"COHERE_API_KEY: {os.environ.get('COHERE_API_KEY')}")

You can display the current configuration uptions picked-up by ReadNext by running:

```bash
readnext config
```

### arxiv-top-categories

The `arxiv-top-categories` command displays the complete list of ArXiv top categories. Note that the categories' keys are case sensitive.

In [ ]:
#| export

@app.command()
def arxiv_top_categories():
    "Display ArXiv main categories. Keys are case sensitive."
    print(main)

You can get the list of all the top categories by using this command line:

```bash 
readnext arxiv-top-categories
```

### arxiv-sub-categories

The `arxiv-sub-categories` command displays the complete list of ArXiv sub categories. Note that the categories' keys are case sensitive.

The arxiv sub categories are:

In [ ]:
#| default_exp main
#| export

@app.command()
def arxiv_sub_categories():
    "Display ArXiv sub categories. Keys are case sensitive."
    print(sub)

You can get the list of all the sub categories by using this command line:

```bash 
readnext arxiv-sub-categories
```

## personalized-papers

The `personalized-papers` command gives a list of personalized papers based on the user's current research focus. That command has two required parameters and three optional:

 - `category` _[required]_ : the ArXiv category to use to query the ArXiv search service. It can be a top or sub category, case sentitive.
 - `focus_collection` _[required]_ : the name of the Zotero collection where all the user's papers of interest are available for ReadNext.
 - `proposals_collection` _[default: ""]_ : the name of the Zotero collection where the papers proposed by ReadNext will be added.
 - `with_artifacts` _[default: False]_ : if set to `True`, the artifacts related to the proposed papers (PDF & summary files) will be added to Zotero.
 - `nb_proposals` _[default: 10]_ : the number of papers that will be proposed by ReadNext.

To get new papers proposals, you have to run the `personalized-papers` command. That command requires two arguments:

 - `category` _[required]_ : the arXiv top, or sub, category from which you want to get new papers proposals
 - `zotero_collection` _[required]_ : the name of the Zotero collection where your papers of interest are stored in Zotero. This is what we refer to as the "Focus" collection above. The name of the collection is case sensitive and should be exactly as written in Zotero.

Then you also have three options available:

 - `--proposals-collection` _[default: ""]_ : which tells ReadNext that you want to save the proposed papers in Zotero, in the Zotero Collection specified by the argument. If you don't use this option, ReadNext will only print the proposed papers in the terminal, but will not save them in Zotero. The default behaviour is that you don't save them in Zotero.
 - `--with-artifacts` / `-a` _[default: False]_ : which tells ReadNext that you want to save the artifacts (PDF file of the papers and their summarization) into Zotero. This is the recommended workflow, but it requires a lot more space in your Zotero account. If you want to do this, you will most likely need to subscribe to one of their paid option.
 - `--nb-proposals` _[default: 10]_ : which tells ReadNext how many papers you want to be proposed.

The following command will propose 3 papers from the `cs.AI` caterory, based on the `Readnext-Focus-LLM` collection in my Zotero library, save them in Zotero in the `Readnext-Propositions-LLM` with all related artifacts:

```sh
readnext personalized-papers cs.AI Readnext-Focus-LLM --proposals-collection=Readnext-Propositions-LLM --with-artifacts --nb-proposals=3
```

As you can see, you can easily create a series of topics you want papers proposals around, where each of the topic is defined by a series of specific papers that you read and found important for your research.


In [ ]:
#| export

@app.command()
def personalized_papers(category: str, 
                        focus_collection: str, 
                        proposals_collection: Annotated[str, 
                                                        typer.Option("--proposals-collection",
                                                        help="Save personalized papers in Zotero in target Zotero collection.")] = "", 
                        with_artifacts: Annotated[bool, 
                                                  typer.Option("--with-artifacts", 
                                                               "-a",
                                                               help="Add paper artifacts (PDFs & summary files) to Zotero when saving.")] = False,                                                               
                        nb_proposals=10):
    """Get personalized papers of a `focus-collection` from an ArXiv `category`. 
    If the category is `all` then all categories that have been locally synced will be used.
    if --proposals-collection is set, then the papers will be uploaded to the 
    that Zotero collection, otherwise it will only be displayed to the command line.
    """

    # Step 1: Make sure the category exists
    if exists(category):
        # Step 2: get today's list of papers from arXiv
        print("[green]Syncing today's ArXiv latest papers...[/green]")
        sync_arxiv(category)

        # Step 3: create embeddings for each of those new papers
        print("[green]Creating embeddings for each new paper...[/green]")
        embed_category_papers(category)

        # Step 4: get personalized papers
        print("[green]Get personalized papers...[/green]")
        ids = get_personalized_papers(category, focus_collection, nb_proposals)

        # Step 5: save personalized papers in Zotero
        if proposals_collection != "":
            print("[green]Saving personalized papers in Zotero...[/green]")
            save_personalized_papers_in_zotero(ids, proposals_collection, with_artifacts)

        # Step 6: display personalized papers to the command line
        search = arxiv.Search(id_list=ids.keys())

        for index, result in enumerate(search.results()):
            print(str(index + 1) + '. [italic yellow][' + list(ids.values())[index] + '][/italic yellow]  [blue][link=' + str(result) + ']' + result.title + '[/link][/blue]')
    else:
        print("[bold red]Error:[/bold red] [italic red]ArXiv category, or sub-category ID non existing.[/italic red] Please specify a valid category ID.")



## Initialize

Before running the command line application, we have to make sure that the tool is properly initialized. The current initialization steps that are required are:

  1. Load environment variables
  2. Make sure that all the configuration options are properly set as environment variables.
  3. Check that all the required local models artifacts are available on the local file system. If not, download them from their source.

In [ ]:
#| export

def config_exists(env_var: str):
    """Check if `env_var` environment variable exists"""
    v = env_var.upper()
    if not os.environ.get(v) or os.environ.get(v) == '':
        print("[bold red]Error:[/bold red] [italic red]Configuration option not set.[/italic red] [yellow]Please set the [bold]" + v + "[/bold] environment variable.[/yellow]\n")

def config_check_one_exists(env_vars: list):
    """Check if one of the `env_vars` environment variables exists"""
    for env_var in env_vars:
        if os.environ.get(env_var.upper()):
            return True
    print("[bold red]Error:[/bold red] [italic red]Configuration option not set.[/italic red] [yellow]Please set one of those [bold]" + repr(env_vars) + "[/bold] environment variables.[/yellow]\n")

One thing that needs to be validated at initialization time is the shape of the embeddings in ChromaDB. If the user changed the setting `EMBEDDING_SYSTEM` from one system to another, then most likely that the number of dimentions will be different. If it is the case, then Chroma won't be able to load the embeddings with a different dimention. This is why we have to warn the user.


In [ ]:
#| export

def get_embeddings_dimensions(chroma_client, category: str):
    """Get the embedding dimensions of the given `category`"""
    return len(chroma_client.get_collection(category).peek(1)['embeddings'][0])

In [ ]:
#| export

def init():
    """Initialize the application"""
    # load environment variables
    load_dotenv()

    # check for the existance of all configuration options
    config_exists('ZOTERO_API_KEY')
    config_exists('ZOTERO_LIBRARY_TYPE')
    config_exists('ZOTERO_LIBRARY_ID')
    config_exists('EMBEDDING_SYSTEM')
    config_exists('DOCS_PATH')
    config_exists('RECOMMENDATIONS_PATH')
    config_check_one_exists(['COHERE_API_KEY', 'MODELS_PATH'])

    chroma_client = chromadb.PersistentClient(path=os.environ.get('CHROMA_DB_PATH'))

    # download models if not already downloaded
    match embedding_system():
        case 'baai-bge-base-en':
           config_exists('MODELS_PATH')
           download_embedding_model(os.environ.get('MODELS_PATH'), 'BAAI/bge-base-en') 
        case 'cohere':
            config_exists('COHERE_API_KEY')
        case other:
            print("[bold red]Error:[/bold red] [italic red]Configuration option not set.[/italic red] [yellow]Please set the [bold]EMBEDDING_SYSTEM[/bold] environment variable to either [bold]BAAI/bge-base-en[/bold] or [bold]cohere[/bold].[/yellow]\n")            

## Entry point

The entry point of the command line interface, the `typer` application will be called to manage the interaction with the users.

In [ ]:
#| export
#| eval: false

if __name__ == "__main__":
    init()
    app()